## 1. raw data 받아오기

In [1]:
import pandas as pd
df = pd.read_csv("naver_shopping_raw_data1.csv")

In [2]:
df.shape

(2400, 14)

## 2. column name  통일하기

In [3]:
# hprice, link 콜럼 지우기
df = df.drop('hprice', axis=1)
df = df.drop('link', axis=1)

# lprice => sale_price 이름 바꾸기
df.rename(columns={'lprice': 'sale_price'}, inplace=True)

# 'apiSource' column 추가하기
df['api_source'] = '네이버'

# title => product_name 이름 바꾸기
df.rename(columns={'title': 'product_name'}, inplace=True)

# productId => product_id 이름 바꾸기
df.rename(columns={'productId': 'product_id'}, inplace=True)

# mallName => mall_name 이름 바꾸기
df.rename(columns={'mallName': 'mall_name'}, inplace=True)


## brand, maker 결측치 제거

In [4]:
# brand, maker 둘 다 데이터가 없을 경우 데이터 삭제
import numpy as np

df['brand'] = df['brand'].replace('', np.nan)
df['maker'] = df['maker'].replace('', np.nan)

df = df.dropna(subset=['brand', 'maker'], how='all')

In [5]:
df.shape

(1912, 13)

In [6]:
# brand는 있고 maker는 없는 경우
df[pd.isna(df['maker'])].groupby(['brand', 'maker'], dropna=False).size().sort_values()

brand  maker
3651   NaN      1
어메이즈핏  NaN      1
앱코     NaN      1
애트놀라   NaN      1
애터미    NaN      1
               ..
니케     NaN      5
이케아    NaN      6
다이소    NaN      6
정직한패드  NaN      7
쉬즈미스   NaN      7
Length: 345, dtype: int64

In [7]:
# maker는 있고 brand는 없는 경우
df[pd.isna(df['brand'])].groupby(['brand', 'maker'], dropna=False).size().sort_values().tail(10)

brand  maker
NaN    메디퓨어     1
       문교       1
       돈키호테     1
       리빙스토리    2
       모나미      2
       해피라인     2
       ALT      2
       엠클로      3
       해인       3
       해피앤퍼피    6
dtype: int64

In [8]:
# 브랜드 종류 개수
df.groupby('brand').size().shape

(812,)

In [9]:
# 메이커 종류 개수
df.groupby('maker').size().shape

(569,)

In [10]:
# brand == nan & maker는 있는 경우, brand에 maker 복붙하기
df.loc[df['brand'].isna(), 'brand'] = df.loc[df['brand'].isna(), 'maker']

In [11]:
# maker == nan & brand는 있는 경우, maker에 brand 복붙하기
df.loc[df['maker'].isna(), 'maker'] = df.loc[df['maker'].isna(), 'brand']

In [12]:
df.shape

(1912, 13)

In [13]:
df.head()

,keyword,product_name,image,sale_price,brand,maker,product_id,category1,category2,category3,category4,mall_name,api_source
0,노트북,삼성 갤럭시북 인강용 사무용 업무용 가성비 윈도우11 저가 싼 태블릿 노트북 추천 기본팩,https://shopping-phinf.pstatic.net/main_879523...,399000,갤럭시북,삼성전자,87952389253,디지털/가전,노트북,NaN,NaN,삼성공식파트너 코인비엠에스,네이버
1,노트북,"삼성전자 갤럭시북4 NT750XGR-A51A 16GB, 256GB",https://shopping-phinf.pstatic.net/main_526312...,682990,갤럭시북4,삼성전자,52631236642,디지털/가전,노트북,NaN,NaN,네이버,네이버
2,노트북,"삼성전자 갤럭시북5 프로 NT960XHA-K52A 32GB, 1TB",https://shopping-phinf.pstatic.net/main_536640...,1975560,갤럭시북5 프로,삼성전자,53664048296,디지털/가전,노트북,NaN,NaN,네이버,네이버
3,노트북,LG전자 울트라PC 라이젠5 사무용 인강용 저렴한 8GB NVMe256GB LG노트북,https://shopping-phinf.pstatic.net/main_837187...,599000,LG전자,LG전자,83718736488,디지털/가전,노트북,NaN,NaN,제이 씨앤에스,네이버
4,노트북,삼성 노트북 갤럭시북 i5 16GB 256GB OS미포함 인강용 사무용 NT550XEZ,https://shopping-phinf.pstatic.net/main_830162...,649000,삼성,삼성전자,83016270660,디지털/가전,노트북,NaN,NaN,스피드썬,네이버


## 4. 카테고리 결측치 통계

- category3 데이터가 없는 경우: 총 60개
- category4 데이터가 없는 경우: 총 753개
- category1, category2는 결측치 없음.

In [14]:
# category3 데이터가 없는 경우
df[pd.isna(df['category3'])].groupby(['category1', 'category2'], dropna=False).size().sort_values()

category1  category2
디지털/가전     노트북          30
           태블릿PC        30
dtype: int64

In [15]:
# category4 데이터가 없는 경우
df[pd.isna(df['category4'])].groupby(['category1', 'category2', 'category3'], dropna=False).size().sort_values()

category1  category2  category3
화장품/미용     마스크/팩      워시오프팩         1
패션의류       여성의류       조끼            1
           남성의류       재킷            3
화장품/미용     향수         남녀공용향수        3
패션의류       여성의류       티셔츠           4
출산/육아      유아동의류      점퍼            4
패션의류       여성의류       트레이닝복         6
디지털/가전     휴대폰        공기계/중고폰       8
패션의류       여성의류       원피스           8
           남성의류       청바지           9
           여성의류       청바지          10
                      점퍼           10
생활/건강      세탁용품       세탁망          13
패션의류       여성의류       바지           13
                      블라우스/셔츠      15
           남성의류       트레이닝복        16
생활/건강      세탁용품       빨래바구니        19
패션의류       여성의류       재킷           21
디지털/가전     휴대폰        자급제폰         21
가구/인테리어    침실가구       서랍장          22
디지털/가전     태블릿PC액세서리  스탠드/Dock     22
식품         과자/베이커리    젤리           23
화장품/미용     향수         남성향수         24
생활/건강      주방용품       도마           24
화장품/미용     마스크/팩      마스크시트        24
           스킨케어   

In [16]:
df.columns

Index(['keyword', 'product_name', 'image', 'sale_price', 'brand', 'maker',
       'product_id', 'category1', 'category2', 'category3', 'category4',
       'mall_name', 'api_source'],
      dtype='object')

In [17]:
df[pd.isna(df['product_name'])].shape

(0, 13)

In [18]:
df.to_csv("naver_shopping_results_final2.csv", index=False, encoding='utf-8-sig')